<a href="https://colab.research.google.com/github/wayne4918/Medical_ChatBot/blob/main/Medical_ChatBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 1.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [32]:
import pandas as pd
from sentence_transformers import SentenceTransformer

# 엑셀 파일 불러오기
df = pd.read_excel('업데이트된_질병.xlsx')

from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('all-MiniLM-L6-v2')

# 증상 문장 데이터를 임베딩 벡터로 변환
symptom_embeddings = model.encode(df['증상'].tolist())



In [33]:
!pip install sentence-transformers faiss-cpu


In [34]:
!pip install langchain langserve


In [35]:
!pip install fastapi uvicorn pydantic


In [36]:
!pip install nest-asyncio


In [37]:
pip install sentence-transformers faiss-cpu


In [38]:
# 데이터프레임 확인
print(df.head())

# 텍스트 데이터를 리스트로 추출
symptoms = df['증상'].tolist()
diseases = df['질병'].tolist()

# 데이터의 형태를 확인합니다.
print(f'Number of symptoms: {len(symptoms)}')
print(f'Number of diseases: {len(diseases)}')


      질병          증상
0  진균 감염    몸이 가려워요.
1  진균 감염   피부가 가려워요.
2  진균 감염    팔이 가려워요.
3  진균 감염   다리가 가려워요.
4  진균 감염  등 쪽이 가려워요.
Number of symptoms: 2847
Number of diseases: 2847


In [39]:
from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('distilbert-base-nli-stsb-mean-tokens')

# 증상 문장을 임베딩 벡터로 변환
embeddings = model.encode(symptoms)

# 변환된 벡터의 형태를 확인
embeddings.shape


(2847, 768)

In [40]:
import faiss
import numpy as np

# 벡터 정규화 (L2 정규화)
normalized_embeddings = embeddings.copy()
faiss.normalize_L2(normalized_embeddings)

# FAISS 인덱스 생성 (내적 기반 유사도)
index_flat = faiss.IndexFlatIP(768)  # 벡터 차원이 768임을 지정
index = faiss.IndexIDMap(index_flat)

# id 배열 생성 (각 벡터의 ID로 사용)
ids = np.arange(len(normalized_embeddings))

# 인덱스에 벡터 추가
index.add_with_ids(normalized_embeddings, ids)

# 인덱스에 추가된 벡터 수 확인
print(f"Number of vectors indexed: {index.ntotal}")


Number of vectors indexed: 2847


In [57]:
def search(query, k=5):
    # 쿼리를 임베딩 벡터로 변환하고 정규화
    query_vector = model.encode([query])
    faiss.normalize_L2(query_vector)

    # FAISS를 사용하여 유사한 벡터 검색
    similarities, ids = index.search(query_vector, k)

    ids = ids[0]
    similarities = similarities[0]

    print(f'Query: {query}\n')

    # 검색된 결과를 데이터프레임으로 정리
    results = df.iloc[ids][['질병', '증상']]
    results['유사도'] = similarities

    return results.reset_index(drop=True)

# 예제 쿼리
result_apple = search('식욕이 없어요', 10)
result_banana = search('피부가 가려워요',10)

print(result_apple)
print()
print(result_banana)

Query: 식욕이 없어요

Query: 피부가 가려워요

          질병          증상      유사도
0         수두  식욕이 사라졌어요.  0.99401
1         수두  식욕이 떨어졌어요.  0.99401
2         수두    식욕이 없어요.  0.99401
3        당뇨병    식욕이 많아요.  0.99401
4  소화성 궤양 질환  식욕이 사라졌어요.  0.99401
5  소화성 궤양 질환  식욕이 떨어졌어요.  0.99401
6  소화성 궤양 질환    식욕이 없어요.  0.99401
7    만성 담즙울체  식욕이 사라졌어요.  0.99401
8    만성 담즙울체  식욕이 떨어졌어요.  0.99401
9    만성 담즙울체    식욕이 없어요.  0.99401

        질병         증상       유사도
0       간염  피부가 가려워요.  0.996255
1       건선  피부가 가려워요.  0.996255
2      여드름  피부가 가려워요.  0.996255
3       수두  피부가 가려워요.  0.996255
4       황달  피부가 가려워요.  0.996255
5    약물 반응  피부가 가려워요.  0.996255
6  만성 담즙울체  피부가 가려워요.  0.996255
7    진균 감염  피부가 가려워요.  0.996255
8       수두   팔이 가려워요.  0.963793
9       황달   팔이 가려워요.  0.963793
